## To save the final results of the behavior for each single animal (Early Lesion)

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
from IPython.display import clear_output, display, HTML, Image
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import time
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib import mlab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec
import matplotlib.backends.backend_pdf

root='/data'
%run Animal_Tags.ipynb
%run BatchRatBehavior.ipynb
%run plotRat_documentation_3_KinematicsInvestigation.ipynb
%run plotRat_documentation_1_GeneralBehavior.ipynb
%run loadRat_documentation.ipynb
%run RunBatchRat_3_CompareGroups.ipynb
%run UtilityTools.ipynb

param={
    "binSize":0.25,
    "trialOffset":20., #max end of trial, in seconds (position will be cutted)
    "sigmaSmoothPosition":0.1,  #smooth the position
    #"sigmaSmoothPosition":0.33 for pavel dataType
    "sigmaSmoothSpeed":0.3, #smooth the speed
    "positionDiffRange": [2.,5.], #min and max differences allowed between two consecutive positions
                                  #min to correct start, max to correct jumps
    "pawFrequencyRange":[2.,10.],
    "startAnalysisParams":[10,0.2,0.5],
    "cameraToTreadmillDelay":2., #seconds, usual time between camera start and treadmill start
    "nbJumpMax" : 100., #if jumps>nbJumpMax, trial is badly tracked
    
    #parameter to detect end of trial (first position minima)
    "endTrial_backPos":55,  # minima is after the animal went once to the back (after first time position>backPos)
    "endTrial_frontPos":30, # minima's position is in front of treadmill (position[end]<frontPos)
    "endTrial_minTimeSec":4, # minima is after minTimeSec seconds (time[end]>minTimeSec)
    }

In [ ]:
def plot_batch_traj(root='/data', axes=None, columnNb=5, animal='Rat166', profile={}, stop_date=''):
    if type(animal) != str:
        raise("enter the name of a single animal")
        
    sessionList=batch_get_session_list(root,animalList=[animal],profile=profile,until_date=stop_date)['Sessions']
    if sessionList:
        if axes is None:
            fig=plt.figure(figsize=(10,12),dpi=150,tight_layout=False)
#             fig.suptitle(animal,y=.92)
            rowNb=(len(sessionList)//columnNb)
            axes=[fig.add_subplot(rowNb,columnNb,i+1) for i,_ in enumerate(sessionList)]
            fig.tight_layout()
    for i,ax in enumerate(axes):
        data=Data(root,sessionList[i][:6],sessionList[i],redoPreprocess=False)
        try:
            plot_positions(data,onlyGood=False,raw=False,vSpan=False,axis=ax,linewidth=.5)
            ax.set_title('Session '+str(i+1))
            ax.set_axis_off()
        except Exception as e:
            print(repr(e))            
    
    return sessionList

### parameters

In [ ]:
if "__file__" not in dir():
    animal='Rat114'
    
    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
#              'Speed':'10',
             'Tag':['Early-Lesion_DLS']
             }
    animalList1=batch_get_animal_list(root,profile1)
    animalList2=batch_get_animal_list(root,profile2)

    TaskParamToPlot=["% good trials","percentile entrance time","standard deviation of entrance time",
        "Forward Speed Vs TreadmillSpeed","Tortuosity"]

    groups={
        "group1":(cm.Greys,"black",animalList1,profile1),
        "group2":(cm.Reds ,"red"  ,animalList2,profile2),
        }

    
    maxNbDays=30
    columnNb=5
    rowNb=(maxNbDays//columnNb)
    histoFileName=[]
    histoFileType=['.tif','.tiff','.jpg','.jpeg']
    n_iteration=1e3
    stop_dayPlot =maxNbDays
    fullLegend=False
    gridX=gridY=15
    figsize=(25,20)
    


In [ ]:
if "__file__" not in dir():
    if animal not in animalList2:
        raise NameError("Bad animal name!")

    # plotting the AllTrajectories
    fig=plt.figure(figsize=figsize,tight_layout=False);
    axes=[]
    for row in range(rowNb):
        for col in range(columnNb):
            axes.append(plt.subplot2grid(shape=(gridX, gridY), loc=(row*2,col*2), colspan=2, rowspan=2, fig=fig))
    plot_batch_traj(root='/data', axes=axes, columnNb=columnNb, animal=animal, profile=profile2)
    
    
    # plotting the histo
    thisRoot="/NAS02"
    all_animals=[os.path.basename(path) for path in sorted(glob.glob(os.path.join(thisRoot,"Rat???")))]
    if all_animals==[]:
        print('NAS02 not mounted!')
        raise()
    animalNasPath=os.path.join(thisRoot, animal, 'Histology')
    if not histoFileName:
        imageFiles=find_file(animalNasPath, extension=histoFileType)
    else:
        imageFiles=[os.path.join(animalNasPath, file) for file in histoFileName]
    
    for i in range(columnNb):
        ax=plt.subplot2grid(shape=(gridX, gridY), loc=(12, i*2), colspan=2, rowspan=3,fig=fig)
        ax.imshow(scipy.ndimage.imread(imageFiles[i]))
        ax.set_title(os.path.basename(imageFiles[i]))
        ax.set_axis_off()
                           
    #plotting the compare groups
    axes=[]
    for i in range(5):
        axes.append(plt.subplot2grid(shape=(gridX, gridY), loc=(i*3,10), colspan=5, rowspan=3,fig=fig))
    
    allResults,colors,colorGroup,ALLobj=plot_mean_animals(root,groups,axes,n_iteration=1e2
                                                          ,parameter={},redo=False,stop_dayPlot=stop_dayPlot
                                                          ,TaskParamToPlot=TaskParamToPlot,fullLegend=False,ylabel='')
    for i,ax in enumerate(axes):
        data=allResults[TaskParamToPlot[i]]['group2'][animal]
        ax.plot(np.arange(stop_dayPlot)+1,data,'s-',color='m',linewidth=3,markersize=9,label=animal)
    ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
        
    fig.tight_layout()
    fig.suptitle(animal,y=1.02,fontsize=15)
    
    #saving the final pdf file
#     pdf = matplotlib.backends.backend_pdf.PdfPages("/home/david/Mostafa/output.pdf")
#     pdf.savefig(fig, bbox_inches='tight')
#     pdf.close()

In [ ]:
fig